In [1]:
import sys, os
sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'src'))
from tqdm import tqdm
from icdlmmeval import util_text
print(sys.path)

%load_ext autoreload

['/home/jovyan/work/icdllmeval/notebooks', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/jovyan/work/icdllmeval/src']


In [2]:
import configparser
import pandas as pd
import json

from icdlmmeval.codiesp.codiformat import CodiFormat
from icdlmmeval.codiesp.prompt_examples import PromptExamples
from icdlmmeval.codiesp import eval
from icdlmmeval.ner_main_predict import NerMainPredict
from icdlmmeval.embedding_lookup import EmbeddingLookup
from icdlmmeval.icd_prompts import IcdPrompts
from icdlmmeval import ner_parsing


codiformat = CodiFormat()


load token-classification pipeline for main terms


2023-12-13 20:48:08.053007: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-13 20:48:08.053078: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-13 20:48:08.053105: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (f1401f85aa9b): /proc/driver/nvidia/version does not exist


In [15]:
split = "test"
file_gpt_descriptions = "/home/jovyan/work/icdllmeval/resources/gpt-descriptions/file-descriptions-test-complete.xlsx"
file_gpt_codes = "/home/jovyan/work/icdllmeval/resources/gpt-codes/predicted-codes-test-complete.xlsx"

In [19]:
# for speed parallel processing has been performed, outputs are merged
code_output = [
    "/home/jovyan/work/icdllmeval/resources/gpt-codes/predicted-codes-test-77-None.xlsx",
    "/home/jovyan/work/icdllmeval/resources/gpt-codes/predicted-codes-test-S0004-06142008000300015-1-S0004-06142005000500011-1.xlsx",
    "/home/jovyan/work/icdllmeval/resources/gpt-codes/predicted-codes-test-S0211-57352011000400009-1-S1131-57682003000300008-1.xlsx"
]

frames = []
unique_files = []
for file in code_output:
    df = pd.read_excel(file)
    df = df[~df['file'].isin(unique_files)]
    unique_files.extend(list(set(df['file'].to_list())))
    frames.append(df)

df_codes = pd.concat(frames)
print(len(df_codes["file"].unique()))

df_codes.head()
df_codes.to_excel(file_gpt_codes)

235


In [17]:
df_descriptions = pd.read_excel(file_gpt_descriptions)
df_codes = pd.read_excel(file_gpt_codes)
df_gold_x = codiformat.get_df_x(split)
# print(df_descriptions.head())
# print(df_codes.head())
# print(df_gold_x.head())


In [18]:
import pandas as pd

unique_files_gold = set(df_gold_x["FILE"].to_list())
unique_files_descriptions = set(df_descriptions["file"].to_list())
unique_files_codes = set(df_codes["file"].to_list())

print("len gold ", len(unique_files_gold))
print("len descriptions ", len(unique_files_descriptions))
print("len codes ", len(unique_files_codes))

missing_descriptions = unique_files_gold.difference(unique_files_descriptions)
missing_codes = unique_files_gold.difference(unique_files_codes)

print("missing descriptions for {} files {}".format(len(missing_descriptions), list(missing_descriptions)))
print("missing codes for {} files {}".format(len(missing_codes), list(missing_codes)))



len gold  250
len descriptions  250
len codes  234
missing descriptions for 0 files []
missing codes for 16 files ['S0378-48352006000300005-1', 'S0210-48062005000100016-1', 'S0211-57352011000400009-1', 'S1137-66272011000300023-1', 'S1130-01082009000300015-1', 'S0210-48062007000500015-1', 'S1131-57682003000300008-1', 'S1699-695X2015000100011-1', 'S0004-06142008000700003-1', 'S0210-56912006000200007-1', 'S0004-06142008000300012-1', 'S0212-71992007000600008-1', 'S0365-66912012000200004-1', 'S0212-16112004000400007-1', 'S0212-16112012000600045-1', 'S0210-48062007001000013-1']


In [30]:
print(list(df_gold_x["FILE"].unique())[35:76])

['S0210-48062005000100016-1', 'S0210-48062005000300014-1', 'S0210-48062005000500016-1', 'S0210-48062005000700010-1', 'S0210-48062005001000013-1', 'S0210-48062006000900012-2', 'S0210-48062006000900014-1', 'S0210-48062007000100004-1', 'S0210-48062007000500015-1', 'S0210-48062007000700015-1', 'S0210-48062007001000013-1', 'S0210-48062009000200017-1', 'S0210-48062009000200018-1', 'S0210-48062009000300013-6', 'S0210-48062009000600016-1', 'S0210-48062009000600017-1', 'S0210-48062009000900015-3', 'S0210-48062009001000016-2', 'S0210-48062010000100019-1', 'S0210-56912006000200007-1', 'S0210-56912006000800008-1', 'S0210-56912008000100008-3', 'S0210-56912008000200007-1', 'S0210-56912008000200007-2', 'S0210-56912008000400007-4', 'S0210-56912009000800006-3', 'S0210-56912010000200009-1', 'S0210-56912011000500009-3', 'S0210-56912012000900010-1', 'S0211-57352011000100008-1', 'S0211-57352011000400009-1', 'S0211-57352013000300012-1', 'S0211-69952011000600018-1', 'S0211-69952012000500025-1', 'S0211-699520

In [36]:
config = configparser.ConfigParser()
config.read('./../resources/config.ini')

path_codiesp_eval = config["codiesp"]['eval']
path_to_codes_D_tsv = path_codiesp_eval + "/codiesp_codes/codiesp-D_codes.tsv"
path_to_codes_P_tsv = path_codiesp_eval + "/codiesp_codes/codiesp-P_codes.tsv"

In [45]:
# EXPLAINABILITY RESULTS

result_name = "166"
code_field = "code_listed" 
df_x_gold, df_x = eval.get_dfs_x_eval(split, df_codes, code_field)

path_x_gold = f"/home/jovyan/work/icdllmeval/resources/gpt-codes/x-gold-{result_name}.tsv"
path_x = f"/home/jovyan/work/icdllmeval/resources/gpt-codes/x-{result_name}.tsv"

codiformat.write_df_csv(path_x_gold, df_x_gold)
codiformat.write_df_csv(path_x, df_x)

path_results = f"/home/jovyan/work/icdllmeval/resources/gpt-codes/x-{result_name}-results.txt"
path_codiesp_eval_script = path_codiesp_eval + "/codiespX_evaluation.py"

!python {path_codiesp_eval_script} -g {path_x_gold} -p {path_x} -cD {path_to_codes_D_tsv} -cP {path_to_codes_P_tsv} > {path_results}

with open(path_results, "r") as file:
  print(file.read())


eval x


/home/jovyan/work/icdllmeval/src/icdlmmeval/codiesp/eval.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  llmcodes_x.rename(columns={'file': 'FILE', 'offsets': 'OFFSETS', 'type': 'TYPE', code_field: 'CODE'}, inplace=True)


/home/jovyan/work/codiesp-evaluation-script/codiespX_evaluation.py:37: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/jovyan/work/codiesp-evaluation-script/codiespX_evaluation.py:92: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/home/jovyan/work/codiesp-evaluation-script/codiespX_evaluation.py:195: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
/home/jovyan/work/codiesp-evaluation-script/codiespX_evaluation.py:206: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
/home/jovyan/work/codiesp-evaluation-script/codiespX_evaluation.py:254: UserWarning: Some documents do not have predicted codes, document-wise Precision not computed for them.
/home/jovyan/work/codiesp-evaluation-script/codiespX_evaluation.py:266: UserWarning: Some docume

In [43]:
# PROCEDURE RESULTS


print(result_name)
code_field = "code_assigned"
df_p_gold, df_p = eval.get_dfs_p_eval(split, df_codes, code_field)

path_p_gold = f"/home/jovyan/work/icdllmeval/resources/gpt-codes/p-gold-{result_name}.tsv"
path_p = f"/home/jovyan/work/icdllmeval/resources/gpt-codes/p-{result_name}.tsv"

codiformat.write_df_csv(path_p_gold, df_p_gold)
codiformat.write_df_csv(path_p, df_p)

path_results_dp = f"/home/jovyan/work/icdllmeval/resources/gpt-codes/p-dp-{result_name}-results.txt"
path_results_f1 = f"/home/jovyan/work/icdllmeval/resources/gpt-codes/p-f1-{result_name}-results.txt"

!python {path_codiesp_eval}/codiespD_P_evaluation.py -g {path_p_gold} -p {path_p} -c {path_to_codes_P_tsv} > {path_results_dp}
!python {path_codiesp_eval}/comp_f1_diag_proc.py -g {path_p_gold} -p {path_p} -c {path_to_codes_P_tsv} > {path_results_f1}

with open(path_results_dp, "r") as file:
  print(file.read())
with open(path_results_f1, "r") as file:
  print(file.read())


166
eval type=PROCEDIMIENTO
/home/jovyan/work/codiesp-evaluation-script/comp_f1_diag_proc.py:110: UserWarning: Some documents do not have predicted codes, document-wise Precision not computed for them.
  warnings.warn('Some documents do not have predicted codes, ' +
/home/jovyan/work/codiesp-evaluation-script/comp_f1_diag_proc.py:133: UserWarning: Some documents do not have predicted codes, document-wise F-score not computed for them.
  warnings.warn('Some documents do not have predicted codes, ' +

MAP estimate: 0.113



-----------------------------------------------------
Clinical case name			Precision
-----------------------------------------------------
S0004-06142005000500011-1		0.333
-----------------------------------------------------
S0004-06142005000900014-1		0.5
-----------------------------------------------------
S0004-06142006000100010-1		0.333
-----------------------------------------------------
S0004-06142006000500012-1		0.0
-------------------------------------------

In [41]:
# DIAGNOSES RESULTS

code_field = "code_assigned"
df_d_gold, df_d = eval.get_dfs_d_eval(split, df_codes, code_field)

path_d_gold = f"/home/jovyan/work/icdllmeval/resources/gpt-codes/d-gold-{result_name}.tsv"
path_d = f"/home/jovyan/work/icdllmeval/resources/gpt-codes/d-{result_name}.tsv"

codiformat.write_df_csv(path_d_gold, df_d_gold)
codiformat.write_df_csv(path_d, df_d)

path_results_dp = f"/home/jovyan/work/icdllmeval/resources/gpt-codes/d-dp-{result_name}-results.txt"
path_results_f1 = f"/home/jovyan/work/icdllmeval/resources/gpt-codes/d-f1-{result_name}-results.txt"

!python {path_codiesp_eval}/codiespD_P_evaluation.py -g {path_d_gold} -p {path_d} -c {path_to_codes_D_tsv} > {path_results_dp}
!python {path_codiesp_eval}/comp_f1_diag_proc.py  -g {path_d_gold} -p {path_d} -c {path_to_codes_D_tsv} > {path_results_f1}

with open(path_results_dp, "r") as file:
  print(file.read())
with open(path_results_f1, "r") as file:
  print(file.read())

eval type=DIAGNOSTICO

MAP estimate: 0.262



-----------------------------------------------------
Clinical case name			Precision
-----------------------------------------------------
S0004-06142005000500011-1		0.75
-----------------------------------------------------
S0004-06142005000900014-1		0.5
-----------------------------------------------------
S0004-06142006000100010-1		0.5
-----------------------------------------------------
S0004-06142006000500012-1		0.471
-----------------------------------------------------
S0004-06142006000600014-1		0.5
-----------------------------------------------------
S0004-06142006000700011-1		0.286
-----------------------------------------------------
S0004-06142006000700012-1		0.182
-----------------------------------------------------
S0004-06142007000100011-1		0.545
-----------------------------------------------------
S0004-06142007000200011-1		0.222
-----------------------------------------------------
S0004-06142007000200017-1		0.455
------